In [2]:
import os
import cv2
import numpy as np
import pandas as pd
import mediapipe as mp
import xgboost as xgb
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tqdm import tqdm

In [3]:
# 1. KONFIGURASI 
IMAGE_DIR = 'AFLW2000/'
LABEL_FILE = 'AFLW2000_labels.csv'
MODEL_OUTPUT_DIR = 'model'

In [4]:
# 2. FUNGSI UNTUK EKSTRAKSI FITUR 
def create_feature_vector(face_landmarks):
    anchor_point = face_landmarks.landmark[1]
    p_left = face_landmarks.landmark[359]
    p_right = face_landmarks.landmark[130]
    scale_distance = np.linalg.norm([p_left.x - p_right.x, p_left.y - p_right.y])
    if scale_distance < 1e-6:
        return None
    feature_vector = []
    for i in range(len(face_landmarks.landmark)):
        if i == 1: continue
        landmark = face_landmarks.landmark[i]
        feature_vector.extend([
            (landmark.x - anchor_point.x) / scale_distance,
            (landmark.y - anchor_point.y) / scale_distance,
            (landmark.z - anchor_point.z) / scale_distance
        ])
    return np.array(feature_vector)

def process_and_prepare_data(image_dir, label_file):
    print("\nMemulai proses ekstraksi fitur dari dataset...")
    labels_df = pd.read_csv(label_file)
    mp_face_mesh = mp.solutions.face_mesh
    face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, min_detection_confidence=0.5, refine_landmarks=False)
    X_data, y_data = [], []
    for index, row in tqdm(labels_df.iterrows(), total=labels_df.shape[0], desc="Processing Images"):
        image_path = os.path.join(image_dir, row['image_name'])
        if not os.path.exists(image_path): continue
        image = cv2.imread(image_path)
        if image is None: continue
        results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                feature_vector = create_feature_vector(face_landmarks)
                if feature_vector is not None:
                    X_data.append(feature_vector)
                    y_data.append([row['pitch'], row['yaw'], row['roll']])
    face_mesh.close()
    print(f"Ekstraksi fitur selesai. Ditemukan {len(X_data)} sampel valid.")
    return np.array(X_data), np.array(y_data)

In [5]:
# 3. FUNGSI BARU (HANYA TRAINING) 

def train_models(X, y, output_dir):
    print("\nMemulai training model XGBoost...")
    os.makedirs(output_dir, exist_ok=True)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    models = {}
    for i, angle in enumerate(['pitch', 'yaw', 'roll']):
        print(f"--- Melatih model untuk {angle.upper()} ---")
        y_train_angle = y_train[:, i]
        model = xgb.XGBRegressor(n_estimators=500, learning_rate=0.05, max_depth=5, random_state=42, n_jobs=-1)
        model.fit(X_train, y_train_angle)
        
        models[angle] = model
        model_path = os.path.join(output_dir, f'xgb_{angle}_model.joblib')
        joblib.dump(model, model_path)
        print(f"Model untuk {angle.upper()} tersimpan di '{model_path}'")
        
    print("\nProses training selesai.")
    # Kembalikan model yang sudah dilatih DAN data uji
    return models, X_test, y_test

In [6]:
#  4. FUNGSI BARU (HANYA EVALUASI)
def evaluate_performance(models, X_test, y_test, tolerance_degrees=5.0):

    print("\n" + "="*45)
    print("--- HASIL EVALUASI MODEL PADA DATA UJI ---")
    print("="*45)
    
    ground_truth = {'pitch': y_test[:, 0], 'yaw': y_test[:, 1], 'roll': y_test[:, 2]}

    for angle in ['pitch', 'yaw', 'roll']:
        model = models[angle]
        y_true = ground_truth[angle]
        y_pred = model.predict(X_test)
        
        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        accuracy_percent = np.sum(np.abs(y_true - y_pred) <= tolerance_degrees) / len(y_true) * 100
        
        print(f"\nAngle: {angle.upper()}")
        print(f"  - Mean Absolute Error (MAE)    : {mae:.2f} derajat")
        print(f"  - Root Mean Squared Error (RMSE) : {rmse:.2f} derajat")
        print(f"  - Akurasi (toleransi ±{tolerance_degrees}°): {accuracy_percent:.2f} %")
    
    print("\n" + "="*45)

In [7]:
# 5. EKSEKUSI UTAMA 

if __name__ == '__main__':
    if not os.path.exists(LABEL_FILE):
        print(f"Error: File label '{LABEL_FILE}' tidak ditemukan.")
    else:
        # Langkah 1: Siapkan data dari file
        X_features, y_labels = process_and_prepare_data(IMAGE_DIR, LABEL_FILE)
        
        if len(X_features) > 0:
            # Langkah 2: Latih model dan dapatkan hasilnya (model & data uji)
            trained_models, X_test_data, y_test_data = train_models(X_features, y_labels, MODEL_OUTPUT_DIR)
            
            # Langkah 3: Evaluasi model menggunakan hasil dari langkah 2
            evaluate_performance(trained_models, X_test_data, y_test_data)
        else:
            print("Tidak ada data yang bisa diproses.")


Memulai proses ekstraksi fitur dari dataset...


Processing Images: 100%|██████████| 2000/2000 [00:25<00:00, 77.65it/s]


Ekstraksi fitur selesai. Ditemukan 1853 sampel valid.

Memulai training model XGBoost...
--- Melatih model untuk PITCH ---
Model untuk PITCH tersimpan di 'model\xgb_pitch_model.joblib'
--- Melatih model untuk YAW ---
Model untuk YAW tersimpan di 'model\xgb_yaw_model.joblib'
--- Melatih model untuk ROLL ---
Model untuk ROLL tersimpan di 'model\xgb_roll_model.joblib'

Proses training selesai.

--- HASIL EVALUASI MODEL PADA DATA UJI ---

Angle: PITCH
  - Mean Absolute Error (MAE)    : 0.14 derajat
  - Root Mean Squared Error (RMSE) : 0.69 derajat
  - Akurasi (toleransi ±5.0°): 99.46 %

Angle: YAW
  - Mean Absolute Error (MAE)    : 0.07 derajat
  - Root Mean Squared Error (RMSE) : 0.16 derajat
  - Akurasi (toleransi ±5.0°): 100.00 %

Angle: ROLL
  - Mean Absolute Error (MAE)    : 0.15 derajat
  - Root Mean Squared Error (RMSE) : 1.18 derajat
  - Akurasi (toleransi ±5.0°): 99.46 %

